In [158]:
import sys
import pandas as pd
from pyspark.sql import SparkSession
from config import data_2010Q1_acquis
from config import data_2010Q1_perf
# create sparksession
spark = SparkSession \
    .builder \
    .appName("insight") \
    .getOrCreate()
acquis = spark.read.option("inferSchema", "true").option("header", False).option("dateFormat", "yyyyMMdd").option("delimiter","|").csv("../realdata/2010Q1/Acquisition_2010Q1.txt")


In [159]:
perf = spark.read.option("inferSchema", "true").option("header", False).option("dateFormat", "yyyyMMdd").option("delimiter","|").csv("../realdata/2010Q1/Performance_2010Q1.txt")

In [160]:
col_names_aquisition = ['loanID', 'originationChannel', 'sellerName', 'origIntRate', 'origUPB', 
                       'origLoanTerm', 'originationDate', 'firstPaymentDate', 'LTV', 'CLTV', 
                        'numOfBorrowers', 'origDebtToIncomeRatio', 'borrowerCredScoreAtOrigination',
                       'firstTimeBuyerIndicator', 'loanPurpose', 'propertyType', 'numOfUnits',
                       'occupancyType', 'propertyState', 'zipCodeShort', 'primaryMortgInsurPercent', 
                       'productType', 'coborrowerCreditScoreAtOrig', 'mortgageInsurType', 
                       'relocationMortgIndicator']
col_names_performance =['loanID', 'monthlyReportingPeriod', 'servicerName', 'currentIntRate',
                       'currentActualUPB', 'loanAge', 'remMonthsToLegalMaturity', 'adjMonthToMaturity',
                       'maturityDate', 'MSA', 'currentLoanDelinqStatus', 'modifFlag', 'zeroBalanceCode',
                       'zeroBalanceEffectiveDate', 'lastPaidInstallDate', 'foreclosureDate',
                       'dispositionDate', 'foreclosureCost', 'propPreservAndReparCosts', 'assetRecoveryCosts',
                       'miscelHoldingExpensesAndCredits', 'associatedTaxesForHoldingProperty', 'netSaleProceeds',
                       'creditEnhacementProceeds', 'repurchaseMakeWholeProceeds','otherForeclosureProceeds',
                       'nonInterestBearingUPB', 'principalForegivenessAmount', 'repurchaseMakeWholeProceedsFlag',
                       'foreclosurePrincipWriteOffAmont', 'servicingActivityIndicator']

In [161]:
i = 0
acquis_rn = acquis
for colname in col_names_aquisition:
    acquis_rn = acquis_rn.withColumnRenamed("_c"+str(i),col_names_aquisition[i])
    i = i + 1
    
i = 0
perf_rn = perf
for colname in col_names_performance:
    perf_rn = perf_rn.withColumnRenamed("_c"+str(i),col_names_performance[i])
    i = i + 1
acquis_rn.select(acquis_rn.columns[:7]).show(5)

+------------+------------------+--------------------+-----------+-------+------------+---------------+
|      loanID|originationChannel|          sellerName|origIntRate|origUPB|origLoanTerm|originationDate|
+------------+------------------+--------------------+-----------+-------+------------+---------------+
|100010079393|                 C|WELLS FARGO BANK,...|      4.875| 284000|         360|        01/2010|
|100013622306|                 R|JPMORGAN CHASE BA...|       4.75|  87000|         180|        12/2009|
|100019943199|                 R|               OTHER|        5.0| 417000|         360|        11/2009|
|100022098429|                 R|               OTHER|       5.25| 461000|         360|        01/2010|
|100023088745|                 R|WELLS FARGO BANK,...|       5.25| 100000|         360|        11/2009|
+------------+------------------+--------------------+-----------+-------+------------+---------------+
only showing top 5 rows



In [82]:
print((acquis_rn.count(), len(acquis_rn.columns)))

(323174, 25)


In [83]:
print(perf_rn.count(),len(perf_rn.columns))

18634553 31


In [84]:
perf_rn.select(perf_rn.columns[:7]).show(10)

+------------+----------------------+--------------------+--------------+----------------+-------+------------------------+
|      loanID|monthlyReportingPeriod|        servicerName|currentIntRate|currentActualUPB|loanAge|remMonthsToLegalMaturity|
+------------+----------------------+--------------------+--------------+----------------+-------+------------------------+
|100010079393|            02/01/2010|WELLS FARGO BANK,...|         4.875|            null|      0|                     360|
|100010079393|            03/01/2010|                null|         4.875|            null|      1|                     359|
|100010079393|            04/01/2010|                null|         4.875|            null|      2|                     358|
|100010079393|            05/01/2010|                null|         4.875|            null|      3|                     357|
|100010079393|            06/01/2010|                null|         4.875|            null|      4|                     356|
|1000100

In [162]:

acquis_rn.createOrReplaceTempView('acquis')
perf_rn.createOrReplaceTempView('perf')

In [163]:
joined = spark.sql(
'''
SELECT perf.loanID, acquis.origIntRate, acquis.origUPB, acquis.originationDate, 
acquis.firstPaymentDate, acquis.LTV, acquis.CLTV, acquis.numOfBorrowers, acquis.origDebtToIncomeRatio, 
acquis.borrowerCredScoreAtOrigination, acquis.zipCodeShort, acquis.primaryMortgInsurPercent, 
perf.monthlyReportingPeriod, perf.loanAge, perf.currentLoanDelinqStatus FROM acquis 
    Inner JOIN perf
        ON acquis.loanID=perf.loanID
'''
)

In [164]:
joined.createOrReplaceTempView('joined')
_2010Q1 = spark.sql(
'''
SELECT * FROM joined
WHERE originationDate IN ("01/2010", "02/2010", "03/2010")
'''
)


In [168]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import IntegerType
_2010Q1 = _2010Q1.withColumn("monthlyReportingPeriod",to_date(_2010Q1.monthlyReportingPeriod,'MM/dd/yyyy'))
_2010Q1 = _2010Q1.withColumn("currentLoanDelinqStatus", _2010Q1["currentLoanDelinqStatus"].cast(IntegerType()))
#_2010Q1.select(_2010Q1.columns[10:15]).show(5)
_2010Q1 = _2010Q1.orderBy(["loanID","originationDate","monthlyReportingPeriod"])

In [174]:
print(_2010Q1.count(), len(_2010Q1.columns))

9567452 15


In [175]:
_2010Q1.select("loanID").distinct().count()

167864

In [180]:
_2010Q1.printSchema()

root
 |-- loanID: long (nullable = true)
 |-- origIntRate: double (nullable = true)
 |-- origUPB: integer (nullable = true)
 |-- originationDate: string (nullable = true)
 |-- firstPaymentDate: string (nullable = true)
 |-- LTV: integer (nullable = true)
 |-- CLTV: integer (nullable = true)
 |-- numOfBorrowers: integer (nullable = true)
 |-- origDebtToIncomeRatio: integer (nullable = true)
 |-- borrowerCredScoreAtOrigination: integer (nullable = true)
 |-- zipCodeShort: integer (nullable = true)
 |-- primaryMortgInsurPercent: integer (nullable = true)
 |-- monthlyReportingPeriod: date (nullable = true)
 |-- loanAge: integer (nullable = true)
 |-- currentLoanDelinqStatus: integer (nullable = true)



In [181]:
pivot_df=_2010Q1.groupBy("loanID").pivot("monthlyReportingPeriod").max("currentLoanDelinqStatus")

In [187]:
pivot_df.select(pivot_df.columns[0:9]).show(8)

+------------+----------+----------+----------+----------+----------+----------+----------+----------+
|      loanID|2010-01-01|2010-02-01|2010-03-01|2010-04-01|2010-05-01|2010-06-01|2010-07-01|2010-08-01|
+------------+----------+----------+----------+----------+----------+----------+----------+----------+
|104858922832|      null|      null|         0|         0|         0|         0|         0|         0|
|110951121377|      null|         0|         0|         0|         0|         0|         0|         0|
|114840283820|      null|         0|         0|         0|         0|         0|         0|         0|
|117516489731|      null|         0|         0|         0|         0|         0|         0|         0|
|123911455425|      null|         0|         0|         0|         0|         0|         0|         0|
|126206350549|      null|         0|         0|         0|         0|         0|         0|         0|
|126899136362|      null|      null|         0|         0|         0|    

In [183]:
pivot_df.select(pivot_df.columns[10:19]).show(3)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|2010-10-01|2010-11-01|2010-12-01|2011-01-01|2011-02-01|2011-03-01|2011-04-01|2011-05-01|2011-06-01|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
only showing top 3 rows



In [184]:
pivot_df.select(pivot_df.columns[20:29]).show(3)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|2011-08-01|2011-09-01|2011-10-01|2011-11-01|2011-12-01|2012-01-01|2012-02-01|2012-03-01|2012-04-01|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
only showing top 3 rows



In [185]:
pivot_df.select(pivot_df.columns[30:39]).show(3)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|2012-06-01|2012-07-01|2012-08-01|2012-09-01|2012-10-01|2012-11-01|2012-12-01|2013-01-01|2013-02-01|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|         0|         0|         0|         0|         0|         0|         0|         0|
|         0|      null|      null|      null|      null|      null|      null|      null|      null|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+
only showing top 3 rows



In [186]:
print(pivot_df.count(), len(pivot_df.columns))

167864 121


In [169]:
_2010Q1.select("loanID","originationDate","monthlyReportingPeriod").show(40)

+------------+---------------+----------------------+
|      loanID|originationDate|monthlyReportingPeriod|
+------------+---------------+----------------------+
|100010079393|        01/2010|            2010-02-01|
|100010079393|        01/2010|            2010-03-01|
|100010079393|        01/2010|            2010-04-01|
|100010079393|        01/2010|            2010-05-01|
|100010079393|        01/2010|            2010-06-01|
|100010079393|        01/2010|            2010-07-01|
|100010079393|        01/2010|            2010-08-01|
|100010079393|        01/2010|            2010-09-01|
|100010079393|        01/2010|            2010-10-01|
|100010079393|        01/2010|            2010-11-01|
|100010079393|        01/2010|            2010-12-01|
|100010079393|        01/2010|            2011-01-01|
|100022098429|        01/2010|            2010-01-01|
|100022098429|        01/2010|            2010-02-01|
|100022098429|        01/2010|            2010-03-01|
|100022098429|        01/201

In [170]:
_2010Q1.select(_2010Q1.columns[10:15]).show(4)

+------------+------------------------+----------------------+-------+-----------------------+
|zipCodeShort|primaryMortgInsurPercent|monthlyReportingPeriod|loanAge|currentLoanDelinqStatus|
+------------+------------------------+----------------------+-------+-----------------------+
|         787|                    null|            2010-02-01|      0|                      0|
|         787|                    null|            2010-03-01|      1|                      0|
|         787|                    null|            2010-04-01|      2|                      0|
|         787|                    null|            2010-05-01|      3|                      0|
+------------+------------------------+----------------------+-------+-----------------------+
only showing top 4 rows



In [171]:
_2010Q1.printSchema()

root
 |-- loanID: long (nullable = true)
 |-- origIntRate: double (nullable = true)
 |-- origUPB: integer (nullable = true)
 |-- originationDate: string (nullable = true)
 |-- firstPaymentDate: string (nullable = true)
 |-- LTV: integer (nullable = true)
 |-- CLTV: integer (nullable = true)
 |-- numOfBorrowers: integer (nullable = true)
 |-- origDebtToIncomeRatio: integer (nullable = true)
 |-- borrowerCredScoreAtOrigination: integer (nullable = true)
 |-- zipCodeShort: integer (nullable = true)
 |-- primaryMortgInsurPercent: integer (nullable = true)
 |-- monthlyReportingPeriod: date (nullable = true)
 |-- loanAge: integer (nullable = true)
 |-- currentLoanDelinqStatus: string (nullable = true)



In [172]:
t=perf_rn.agg({"monthlyReportingPeriod":"max"}).collect()[0]
print(t)

Row(max(monthlyReportingPeriod)='12/01/2019')


In [155]:
print(perf_rn.count())

18634553


In [173]:
t=perf_rn.agg({"monthlyReportingPeriod":"min"}).collect()[0]
print(t)

Row(min(monthlyReportingPeriod)='01/01/2010')
